In [1]:
import sys
sys.path.insert(0, '..')

# Граф

In [2]:
import pandas as pd

from src.eventstream.eventstream import Eventstream
from src.eventstream.schema import RawDataSchema, EventstreamSchema
from src.graph.p_graph import PGraph, EventsNode

raw_data = pd.read_csv('simple-onlineshop.csv')

raw_data_schema = RawDataSchema(
                            event_name="event",
                            event_timestamp="timestamp",
                            user_id="user_id")

source = Eventstream(
    raw_data=raw_data,
    raw_data_schema=raw_data_schema,
    schema=EventstreamSchema()
)

graph = PGraph(source_stream=source)

In [3]:
source_df = source.to_dataframe()
source_df['event_name'].value_counts()

catalog             14518
main                 5635
lost                 3098
cart                 2842
product2             2172
delivery_choice      1686
product1             1515
payment_choice       1107
delivery_courier      834
payment_done          706
payment_card          565
delivery_pickup       506
payment_cash          197
Name: event_name, dtype: int64

In [22]:
all(i in source_df.columns for i in ['event_id', 'ccc'])
source_df.columns[-1]

'user_id'

In [14]:
source_df.columns

Index(['event_id', 'event_type', 'event_index', 'event_name',
       'event_timestamp', 'user_id'],
      dtype='object')

# truncate path

## Описание работы функции

Датапроцессор удаляет часть траектории относительно заданного события
Параметры:
```drop_before``` - событие, до которого удаляется часть траектории пользователя (само событие остается)
```drop_after``` -событие, после которого удаляется часть траектории пользователя (само событие остается)
```occurrence_before``` и ```occurrence_after``` - Будет применяться, если в траектории несколько событий, которые мы выбрали как целевые для обрезки. Принимают значение first (обрезка проведется по событию с минимальным timestamp в траектории пользователя) или last (по событию с максимальным timestamp в траектории пользователя).
```shift_before``` и ```shift_after``` - задается целым числом, обозначает сдвиг на заданное количество шагов относительного события из параметра "drop_*" для каждого типа обрезки
(по дефолту - 0, отрицательное - сдвиг влево по оси времени, положительное - сдвиг вправо по оси времени)


In [7]:
%%time
from src.data_processors_lib.rete import TruncatePath, TruncatePathParams
params = {
            'drop_before' : 'product2',
            'drop_after' : None,
            'occurrence_before' : 'first',
            'occurrence_after' : 'last',
            'shift_before' : 0,
            'shift_after' : 0
}

truncate_path = EventsNode(
    TruncatePath(params=TruncatePathParams(**params)))

graph.add_node(
    node=truncate_path,
    parents=[graph.root]
)

result = graph.combine(
    node=truncate_path
)
result.to_dataframe()

,event_id,event_type,event_index,event_name,event_timestamp,user_id
0,aae373f5-587a-4cbc-86f1-0d1b1daad436,raw,0,catalog,2019-11-01 17:59:13.273932,219483890.0
1,2a273dd5-2d4d-49c4-a8b9-6631faf7594f,raw,1,product1,2019-11-01 17:59:28.459271,219483890.0
2,27919e26-9063-4e1a-bd44-9b7fa579baae,raw,2,cart,2019-11-01 17:59:29.502214,219483890.0
3,2349d3f5-e7b8-4a1b-aae4-4d0dffbd955c,raw,3,catalog,2019-11-01 17:59:32.557029,219483890.0
44,3def435d-66b0-49ab-95c6-f4aa3ccb3fef,raw,44,product1,2019-11-01 22:33:19.961115,629881394.0
...,...,...,...,...,...,...
44417,c708c296-8853-42bb-b119-12dee7cd6e56,raw,44417,catalog,2020-04-29 12:47:40.975732,501098384.0
44418,ada68fcc-8b0a-45b1-a8cd-ddda1bc8cd73,raw,44418,catalog,2020-04-29 12:48:01.809577,501098384.0
44419,a44fcc6f-41cb-4568-97b7-157cab796296,raw,44419,main,2020-04-29 12:48:01.938488,501098384.0
44420,f54ceb51-68f2-46d9-8015-bc5139a0a2ca,raw,44420,catalog,2020-04-29 12:48:06.595390,501098384.0


In [5]:
# %%time
# from src.data_processors_lib.rete import CutPathBeforeEvent, CutPathBeforeEventParams
# params = {
#     'cutoff_events': ['product2'],
#     'cut_shift': 0,
#     'min_cjm': 0
# }
#
# cut_before = EventsNode(
#     CutPathBeforeEvent(params=CutPathBeforeEventParams(**params)))
#
# graph.add_node(
#     node=cut_before,
#     parents=[graph.root]
# )
#
# result = graph.combine(
#     node=cut_before
# )


/Users/Daria/Dev/Retentioneering/retentioneering-tools-new-arch/examples/../src/data_processors_lib/rete/cut_path_before_event.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cut["num_groups"] = df_cut.groupby([user_col])[time_col].transform(
/Users/Daria/Dev/Retentioneering/retentioneering-tools-new-arch/examples/../src/data_processors_lib/rete/cut_path_before_event.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_to_del["ref"] = df_to_del[eventstream.schema.event_id]


CPU times: user 44.6 s, sys: 190 ms, total: 44.8 s
Wall time: 45.5 s


In [23]:
occurrence_before = 0
not occurrence_before

True